In [ ]:
from importlib import reload
#---------------------------------------------------------------------
import sys, os
import pandas as pd

#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
#-----
from DOVSAudit import DOVSAudit
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())

In [ ]:
#--------------------------------------------------
# NOTE: opco is not absolutely necessary, but highly suggested
#       Running data grabs is much more efficient with OPCO, and, if you don't use OPCO frequently
#         you will be contacted by Cloud Computing
#       If one does not want to use opco, simply set opco = None below
#-------------------------
# outg_rec_nb = '14318196'
# opco        = 'oh'

# outg_rec_nb = '14344224'
# opco        = 'ap'

outg_rec_nb = '14348865'
opco        = 'ap'

In [ ]:
#--------------------------------------------------
calculate_by_PN               = True
combine_by_PN_likeness_thresh = pd.Timedelta('15 minutes')
expand_outg_search_time_tight = pd.Timedelta('1 hours')
expand_outg_search_time_loose = pd.Timedelta('12 hours')
use_est_outg_times            = False
use_full_ede_outgs            = False
daq_search_time_window        = pd.Timedelta('24 hours')
#-------------------------
run_outg_inclusion_assessment = True
max_pct_PNs_missing_allowed   = 0
#-------------------------
n_PNs_w_power_threshold       = 95
#-------------------------
#-------------------------
include_suboutg_endpt_plots   = True

In [ ]:
#--------------------------------------------------
audit_i = DOVSAudit(
    outg_rec_nb                   = outg_rec_nb, 
    opco                          = opco, 
    calculate_by_PN               = calculate_by_PN, 
    combine_by_PN_likeness_thresh = combine_by_PN_likeness_thresh, 
    expand_outg_search_time_tight = expand_outg_search_time_tight, 
    expand_outg_search_time_loose = expand_outg_search_time_loose, 
    use_est_outg_times            = use_est_outg_times, 
    use_full_ede_outgs            = use_full_ede_outgs, 
    daq_search_time_window        = daq_search_time_window, 
)

#-------------------------
audit_i.build_basic_data(
    slicers_ami                   = None, 
    build_sql_fncn_ami            = AMINonVee_SQL.build_sql_usg, 
    addtnl_build_sql_kwargs_ami   = None, 
    run_std_init_ami              = True, 
    save_args_ami                 = False, 
    #-----
    build_sql_fncn_ede            = AMIEndEvents_SQL.build_sql_end_events, 
    addtnl_build_sql_kwargs_ede   = None, 
    pdpu_only                     = True, 
    run_std_init_ede              = True, 
    save_args_ede                 = False, 
    #-----
    dovs_df                       = None, 
    assert_outg_rec_nb_in_dovs_df = True, 
    #-----
    mp_df                         = None, 
    mp_df_outg_rec_nb_col         = 'OUTG_REC_NB', 
    #-----
    drop_mp_dups_fuzziness        = pd.Timedelta('1 hour'), 
    addtnl_mp_df_cols             = ['technology_tx'], 
    assert_all_PNs_found          = True, 
    consolidate_PNs_batch_size    = 1000, 
    early_return                  = False, 
    
)

#-------------------------
run_result = audit_i.run_audit(
    run_outg_inclusion_assessment = run_outg_inclusion_assessment, 
    max_pct_PNs_missing_allowed   = max_pct_PNs_missing_allowed, 
)

In [ ]:
run_result

In [ ]:
audit_i.best_ests_df

In [ ]:
#--------------------------------------------------
detailed_summary_df_i, detailed_summary_df_dovs_beg_i = audit_i.get_detailed_summary_df_and_dovs_beg(
    delta_t_off_cut = pd.Timedelta('5min'), 
    delta_t_on_cut  = pd.Timedelta('5min'), 
    delta_ci_cut    = 3, 
    delta_cmi_cut   = None, 
    n_PNs_w_power_threshold = n_PNs_w_power_threshold, 
)

In [ ]:
detailed_summary_df_i

In [ ]:
detailed_summary_df_dovs_beg_i

In [ ]:
ci_cmi_summary_df_i = audit_i.get_ci_cmi_summary(return_type = pd.DataFrame)
ci_cmi_summary_df_i

In [ ]:
fig_num = 0
#--------------------------------------------------
fig, axs = audit_i.plot_results(
    include_dovs_beg_text      = True, 
    name                       = 'AMI', 
    expand_time                = pd.Timedelta('1 hour'), 
    n_PNs_w_power_threshold    = n_PNs_w_power_threshold, 
    fig_num                    = fig_num
)
fig_num += 1

In [ ]:
fig, axs = audit_i.plot_results_dovs_beg(
    include_full_alg_text      = True, 
    name                       = 'AMI w/ DOVS t_beg', 
    expand_time                = pd.Timedelta('1 hour'), 
    n_PNs_w_power_threshold    = n_PNs_w_power_threshold, 
    fig_num                    = fig_num
)
fig_num += 1

In [ ]:
if include_suboutg_endpt_plots:
    fig_axs = audit_i.plot_zoomed_endpts(
        fig_num     = fig_num
    )
    fig_num += 1

In [ ]:
audit_i.best_ests_df

In [ ]:
audit_i.best_ests_df_w_keep_info

In [ ]:
# audit_i.best_ests_df_w_keep_info.to_pickle(r'C:\Users\s346557\Downloads\we.pkl')

In [ ]:
audit_i.best_ests_df.head(1)

In [ ]:
audit_i.best_ests_df_w_keep_info

In [ ]:
# audit_i.best_ests_df_w_keep_info['resolved_details'].unique().tolist()

In [ ]:
print(audit_i.generate_warnings_text())

In [ ]:
audit_i.warnings_dict